This notebook explores methods for calculating MLD globally on llc4320 data

In [26]:
import xarray as xr
import matplotlib.pyplot as plt
import zarr
import numpy as np
import dask
import dask.array as da
import numcodecs

from fastjmd95 import jmd95numba 

open LLC4320, make a skeleton zarr file for future MLD writing

In [27]:
t_0 = 432
t_1 = t_0 + (2 * 5 * 24)#(365*24)
LLC = xr.open_zarr('/orcd/data/abodner/003/LLC4320/LLC4320',consolidated=False)[['Theta', 'Salt', 'rA', 'Z','XC','YC']].isel(time=slice(t_0,t_1),face=[7,10])

In [29]:
# native horizontal sizes
nt = LLC.dims["time"]
nf = LLC.dims["face"]
nj = LLC.dims["j"]
ni = LLC.dims["i"]

# use Dask to create a LAZY array (no memory allocation)
MLD_lazy = da.empty(
    shape=(nt, nf, nj, ni),
    chunks=(1, 1, 720, 720),
    dtype=np.float32,
)

MLD_ds = xr.Dataset(
    {
        "MLD": (("time", "face", "j", "i"), MLD_lazy)
    },
    coords={
        "time": LLC.time,
        "face": LLC.face,
        "j": LLC.j,
        "i": LLC.i,
        "XC": LLC.XC,
        "YC": LLC.YC,
    },
)

compressor = numcodecs.Blosc(cname='zstd', clevel=3, shuffle=numcodecs.Blosc.BITSHUFFLE)

encoding = {
    "MLD": {
        "chunks": (1, 1, 720, 720),
        "compressor": compressor,
        "dtype": "float32",
        "_FillValue": np.nan,
    }
}


MLD_ds.to_zarr(
    "/orcd/data/abodner/002/cody/MLD_llc4320/MLD_ds_test.zarr",
    mode="w",
    encoding=encoding,
    consolidated=True,
    zarr_format=2,
    compute=False
)

/tmp/ipykernel_2143994/4108870245.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nt = LLC.dims["time"]
/tmp/ipykernel_2143994/4108870245.py:3: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nf = LLC.dims["face"]
/tmp/ipykernel_2143994/4108870245.py:4: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nj = LLC.dims["j"]
/tmp/ipykernel_2143994/4108870245.py:5: FutureWarning: The return type of

Delayed('_finalize_store-d5632215-9205-4a1e-a4ed-70f3860110a4')

In [30]:
MLD_test = xr.open_zarr('/orcd/data/abodner/002/cody/MLD_llc4320/MLD_ds_test.zarr',consolidated=False)

In [31]:
MLD_test

<xarray.Dataset> Size: 36GB
Dimensions:  (time: 240, face: 2, j: 4320, i: 4320)
Coordinates:
    XC       (face, j, i) float32 149MB dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
  * j        (j) int16 9kB 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
  * i        (i) int16 9kB 0 1 2 3 4 5 6 ... 4313 4314 4315 4316 4317 4318 4319
    YC       (face, j, i) float32 149MB dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
  * time     (time) datetime64[ns] 2kB 2011-10-01 ... 2011-10-10T23:00:00
  * face     (face) int16 4B 7 10
Data variables:
    MLD      (time, face, j, i) float32 36GB dask.array<chunksize=(1, 1, 720, 720), meta=np.ndarray>